In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os, tempfile, glob, random, json
from pathlib import Path
from IPython.display import Markdown
from PIL import Image
from getpass import getpass
import numpy as np
from itertools import combinations
import tiktoken
from uuid import uuid4
# Embedding model
from FlagEmbedding import BGEM3FlagModel

# LLM: openai and google_genai
import openai
from langchain_openai import OpenAI, OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# LLM: HuggingFace
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.llms import HuggingFaceHub

# LLM: Ollama
from langchain_ollama import ChatOllama

# langchain prompts, memory, chains...
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.prompts.base import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.messages import ChatMessage, HumanMessage
from langchain.embeddings.base import Embeddings

# Document loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
    JSONLoader
)

# Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

# OutputParser
from langchain_core.output_parsers import StrOutputParser

# Chroma: vectorstore
from langchain_community.vectorstores import Chroma

# Contextual Compression

#from langchain.retrievers import DocumentCompressorPipeline

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder

#from langchain.retrievers import EmbeddingsFilter
#from langchain.retrievers import ContextualCompressionRetriever

# Cohere
from langchain_community.llms import Cohere

### Creamos la clase necesaria para poder trabajar con el modelo de embedding

In [2]:
class BGEM3Langchain(Embeddings):
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        results = self.model.encode(texts, return_dense=True, return_sparse=False, return_colbert_vecs=False)
        # Solamente devolvemos el embedding por densidad, ya que, estos son los que funcionan en chromadb
        return results["dense_vecs"].tolist()

    def embed_query(self, text):
        result = self.model.encode([text], return_dense=True, return_sparse=False, return_colbert_vecs=False)
        return result["dense_vecs"][0].tolist()

### Cargamos el JSON CPEP

In [3]:
with open("../resources/cpe.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []

In [4]:
for item in data:
    tipo = item.get("tipo", "").capitalize()

    if tipo == "Introduccion":
        text = f"Título: {item.get('titulo', '')}\n" \
                f"Subtítulo: {item.get('subtitulo', '')}\n\n" \
                f"{item.get('contenido', '')}"

    elif tipo == "Articulo":
        text = (
            f"Parte {item.get('parte_num', '')}: {item.get('parte_nom', '')}\n"
            f"Título {item.get('titulo_num', '')}: {item.get('titulo_nom', '')}\n"
            f"Capítulo {item.get('capitulo_num', '')}: {item.get('capitulo_nom', '')}\n"
            f"Sección {item.get('seccion_num', '')}: {item.get('seccion_nom', '')}\n"
            f"Artículo {item.get('art_num', '')}: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )
        
    elif tipo == "Disposición":
        text = (
            f"Disposición {item.get('disposicion', '')}\n"
            f"Nombre jurídico: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )

    # Pongo este else, porque sino me sale error en page_content=text, porque es posible que text este vacio
    else:
        text = item.get("contenido", "")

    docs.append(Document(page_content=text, metadata={"tipo": tipo}))

print(f"Se cargaron {len(docs)} documentos desde el JSON.")

Se cargaron 426 documentos desde el JSON.


In [5]:
import random
random_document_id = random.choice(range(len(docs)))

print("test: ", random_document_id)
print(docs[random_document_id])

test:  156
page_content='Parte Segunda: Estructura y organización funcional del estado
Título 1: Órgano legislativo
Capítulo Primero: Composición y atribuciones de la asamblea legislativa plurinacional
Sección None: None
Artículo 155: Inauguración de Sesiones de la Asamblea Legislativa Plurinacional

La Asamblea Legislativa Plurinacional inaugurará sus sesiones el 6 de Agosto en la Capital de Bolivia, salvo convocatoria expresa de su Presidenta o Presidente.' metadata={'tipo': 'Articulo'}


In [6]:
# Embedding Model
model_embedding = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
embedding_adapter = BGEM3Langchain(model_embedding)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
LOCAL_VECTOR_STORE_DIR = Path("./data").resolve().parent.joinpath("data", "vector_stores")

In [8]:
def create_vectorstore(collection_name, model_embedding, vectorstore_name):
    persist_directory = ("./" + vectorstore_name)
    vector_store = Chroma(
        collection_name = collection_name,
        embedding_function=model_embedding,
        persist_directory = persist_directory
    )
    return vector_store, vectorstore_name

In [16]:
create_new_vectorstore = True # Cambiar a true si queremos volver a crear la bd vectorial
if create_new_vectorstore:
    vector_store_bgem3,_ = create_vectorstore(
        collection_name = "cpep_bgem3",
        model_embedding = embedding_adapter,
        vectorstore_name= "vectordb/cpep_bgem3"
    )
    print("vector_store_bgem3:",vector_store_bgem3._collection.count(),"documentos.")

C:\Users\mauri\AppData\Local\Temp\ipykernel_19172\739684382.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store = Chroma(


vector_store_bgem3: 0 documentos.


In [10]:
# Ids aleatorios evita colisiones, pero no tiene significado semantico
uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store_bgem3.add_documents(documents=docs, ids=uuids)
print("vector_store_bgem3:",vector_store_bgem3._collection.count(),"documentos.")

NameError: name 'vector_store_bgem3' is not defined

In [11]:
ids = []
metas = []
cont = 0

for i in range(len(docs)):
    if (i<2):
        ids.append(f"introduccion_{i+1}")
        metas.append({"tipo": "introduccion", "número": i+1})
    elif(i<399):
        ids.append(f"artículo_{i-1}")
        metas.append({"tipo": "artículo", "número": i-1})
    elif(i==399):
        ids.append(f"disposición_398_A")
        metas.append({"tipo": "disposicion", "número": "398_A"})
    elif(i==400):
        ids.append(f"disposición_398_B")
        metas.append({"tipo": "disposicion", "número": "398_B"})
    elif(i<414):
        ids.append(f"artículo_{i-2}")
        metas.append({"tipo": "artículo", "número": i-2})
    else:
        cont += 1
        ids.append(f"disposición_{cont}")
        metas.append({"tipo": "disposicion", "número": cont})

assert len(docs) == len(ids) == len(metas)

In [17]:
vector_store_bgem3.add_documents(documents=docs, ids=ids)
print("vector_store_bgem3:",vector_store_bgem3._collection.count(),"documentos.")

pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 27.44it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 2/2 [00:03<00:00,  1.56s/it]


vector_store_bgem3: 426 documentos.


In [18]:
print(vector_store_bgem3._collection.count())
print(vector_store_bgem3._collection.peek())

426
{'ids': ['introduccion_1', 'introduccion_2', 'artículo_1', 'artículo_2', 'artículo_3', 'artículo_4', 'artículo_5', 'artículo_6', 'artículo_7', 'artículo_8'], 'embeddings': array([[ 0.04119873,  0.03131104, -0.01198578, ...,  0.01422882,
        -0.00476074, -0.02758789],
       [ 0.037323  ,  0.00144005, -0.04879761, ...,  0.00862885,
         0.00226021, -0.008255  ],
       [ 0.05194092, -0.00133228, -0.01771545, ...,  0.00419235,
         0.00993347, -0.03488159],
       ...,
       [ 0.05209351,  0.03167725, -0.04049683, ...,  0.00888824,
         0.04806519, -0.01847839],
       [ 0.03866577,  0.02000427, -0.02864075, ...,  0.01675415,
        -0.01379395, -0.02059937],
       [ 0.01986694, -0.00522232,  0.01384735, ...,  0.00788879,
         0.02572632, -0.037323  ]], shape=(10, 1024)), 'documents': ['Título: Antecedentes Legales\nSubtítulo: None\n\nLa Constitución Política del Estado, promulgada el 7 de febrero de 2009, consta de 5 Partes: I. BASES FUNDAMENTALES DEL ESTADO I

### Similarity Search

In [19]:
def print_documents(docs,search_with_score=False):
    if search_with_score:
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc[0].page_content +"\n\nscore:"+str(round(doc[-1],3))+"\n" 
                for i, doc in enumerate(docs)]
            )
        )
    else:
        # used for similarity_search or max_marginal_relevance_search
        print(
            f"\n{'-' * 100}\n".join(
                [f"Document {i+1}:\n\n" + doc.page_content 
                for i, doc in enumerate(docs)]
            )
        )  

In [20]:
query = '¿Qué dice el artículo 1 de la constitución política del estado Boliviano?'
docs_withScores = vector_store_bgem3.similarity_search_with_score(query,k=5)

print_documents(docs_withScores,search_with_score=True)

Document 1:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Primero: Modelo de Estado
Sección None: None
Artículo 1: Modelol de Estado

Bolivia se constituye en un Estado Unitario Social de Derecho Plurinacional Comunitario, libre, independiente, soberano, democrático, intercultural, descentralizado y con autonomías. Bolivia se funda en la pluralidad y el pluralismo político, económico, jurídico, cultural y lingüístico, dentro del proceso integrador del país.

score:0.77

----------------------------------------------------------------------------------------------------
Document 2:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Segundo: Principios, Valores y Fines del Estado
Sección None: None
Artículo 10: Carácter Pacifista del Estado

I. Bolivia es un Estado pacifista, que promueve la cultura de la paz y el derecho a la paz, a

In [21]:
query_embeddings = embedding_adapter.embed_query(query)
docs_embeddings = embedding_adapter.embed_documents(
    [docs_withScores[i][0].page_content 
    for i in range(len(docs_withScores))
    ]
)

for i in range(len(docs_embeddings)):
    dot_product = round(np.dot(query_embeddings, docs_embeddings[i]),4)
    print(f"Similarty of document_{i} to the query: {dot_product}")

Similarty of document_0 to the query: 0.6149
Similarty of document_1 to the query: 0.5992
Similarty of document_2 to the query: 0.5826
Similarty of document_3 to the query: 0.5797
Similarty of document_4 to the query: 0.5777


### Maximum marginal relevance search (MMR) search

In [22]:
query = '¿Qué dice el artículo 1 de la constitución política del estado Boliviano?'
docs_MMR = vector_store_bgem3.max_marginal_relevance_search(query,k=4)

print_documents(docs_MMR)

Document 1:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Primero: Modelo de Estado
Sección None: None
Artículo 1: Modelol de Estado

Bolivia se constituye en un Estado Unitario Social de Derecho Plurinacional Comunitario, libre, independiente, soberano, democrático, intercultural, descentralizado y con autonomías. Bolivia se funda en la pluralidad y el pluralismo político, económico, jurídico, cultural y lingüístico, dentro del proceso integrador del país.
----------------------------------------------------------------------------------------------------
Document 2:

Parte Segunda: Estructura y organización funcional del estado
Título 7: Fuerzas Armadas y Policía Boliviana
Capítulo Primero: Fuerzas Armadas
Sección None: None
Artículo 249: Servicio Militar Obligatorio

Todo boliviano estará obligado a prestar servicio militar, de acuerdo con la ley.
--------------------------------------------------------

## Retrievers

### Vectorstore-backed retriever

In [23]:
def Vectorstore_backed_retriever(vectorstore,search_type="similarity",k=4,score_threshold=None):
    """create a vectorsore-backed retriever
    Parameters: 
        search_type: Defines the type of search that the Retriever should perform.
            Can be "similarity" (default), "mmr", or "similarity_score_threshold"
        k: number of documents to return (Default: 4) 
        score_threshold: Minimum relevance threshold for similarity_score_threshold (default=None)
    """
    search_kwargs={}
    if k is not None:
        search_kwargs['k'] = k
    if score_threshold is not None:
        search_kwargs['score_threshold'] = score_threshold

    retriever = vectorstore.as_retriever(
        search_type=search_type,
        search_kwargs=search_kwargs
    )
    return retriever

In [24]:
# similarity search
base_retriever_bgem3 = Vectorstore_backed_retriever(vector_store_bgem3, "similarity", k=5)
#base_retriever_OpenAI = Vectorstore_backed_retriever(vector_store_OpenAI,"similarity",k=10)
#base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"similarity",k=10)
#base_retriever_HF = Vectorstore_backed_retriever(vector_store_HF,"similarity",k=10)

# # MMMR seach
# base_retriever_OpenAI = Vectorstore_backed_retriever(vector_store_OpenAI,"mmr",k=10)
# base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"mmr",k=10)
# base_retriever_HF = Vectorstore_backed_retriever(vector_store_HF,"mmr",k=10)

# # similarity_score_threshold search
# base_retriever_OpenAI = Vectorstore_backed_retriever(vector_store_OpenAI,"similarity_score_threshold",score_threshold=0.5,k=10)
# base_retriever_google = Vectorstore_backed_retriever(vector_store_google,"similarity_score_threshold",score_threshold=0.5,k=10)
# base_retriever_HF = Vectorstore_backed_retriever(vector_store_HF,"similarity_score_threshold",score_threshold=0.5,k=10)

In [25]:
# Get relevant documents
query = '¿Qué dice el artículo 1 de la constitución política del estado Boliviano?'
relevant_docs = base_retriever_bgem3.invoke(query)

print_documents(relevant_docs)

Document 1:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Primero: Modelo de Estado
Sección None: None
Artículo 1: Modelol de Estado

Bolivia se constituye en un Estado Unitario Social de Derecho Plurinacional Comunitario, libre, independiente, soberano, democrático, intercultural, descentralizado y con autonomías. Bolivia se funda en la pluralidad y el pluralismo político, económico, jurídico, cultural y lingüístico, dentro del proceso integrador del país.
----------------------------------------------------------------------------------------------------
Document 2:

Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 1: Bases fundamentales del estado
Capítulo Segundo: Principios, Valores y Fines del Estado
Sección None: None
Artículo 10: Carácter Pacifista del Estado

I. Bolivia es un Estado pacifista, que promueve la cultura de la paz y el derecho a la paz, así como la co

## Retrieval: put it all together

In [ ]:
def retrieval_blocks(
    create_vectorstore=False,# if True a Chroma vectorstore is created, else the Chroma vectorstore will be loaded
    LLM_service="mistral:7b-instruct-v0.2-q8_0",
    vectorstore_name="vectordb/cpep_bgem3",
    #chunk_size = 1600, chunk_overlap=200, # parameters of the RecursiveCharacterTextSplitter
    retriever_type="Vectorstore_backed_retriever",
    base_retriever_search_type="similarity", base_retriever_k=5, base_retriever_score_threshold=None,
    #compression_retriever_k=16,
    #cohere_api_key="***", cohere_model="rerank-multilingual-v2.0", cohere_top_n=8,
):
    """
    Rertieval includes: document loaders, text splitter, vectorstore and retriever. 
    
    Parameters: 
        create_vectorstore (boolean): If True, a new Chroma vectorstore will be created. Otherwise, an existing vectorstore will be loaded.
        LLM_service: OpenAI, Google or HuggingFace.
        vectorstore_name (str): the name of the vectorstore.
        chunk_size and chunk_overlap: parameters of the RecursiveCharacterTextSplitter, default = (1600,200).
        
        retriever_type (str): in [Vectorstore_backed_retriever,Contextual_compression,Cohere_reranker]
        
        base_retriever_search_type: search_type in ["similarity", "mmr", "similarity_score_threshold"], default = similarity.
        base_retriever_k: The most similar vectors to retrieve (default k = 10).  
        base_retriever_score_threshold: score_threshold used by the base retriever, default = None.

        compression_retriever_k: top k documents returned by the compression retriever, default=16
        
        cohere_api_key: Cohere API key
        cohere_model (str): The Cohere model can be either 'rerank-english-v2.0' or 'rerank-multilingual-v2.0', with the latter being the default.
        cohere_top_n: top n results returned by Cohere rerank, default = 8.
    Output:
        retriever.
    """
    try:
        # Create new Vectorstore (Chroma index)
        if create_vectorstore: 
            # 1. load documents
            documents = langchain_document_loader(TMP_DIR)
            
            # 2. Text Splitter: split documents to chunks
            '''text_splitter = RecursiveCharacterTextSplitter(
                separators = ["newline", "newline", " ", ""],    
                chunk_size = chunk_size,
                chunk_overlap= chunk_overlap
            )'''
            #chunks = text_splitter.split_documents(documents=documents)
            
            # 3. Embeddings
            embeddings = select_embeddings_model(LLM_service=LLM_service)
        
            # 4. Vectorsore: create Chroma index
            vector_store = create_vectorstore(
                embeddings=embeddings,
                documents = chunks,
                vectorstore_name=vectorstore_name,
            )
    
        # 5. Load a Vectorstore (Chroma index)
        else: 
            embeddings = select_embeddings_model(LLM_service=LLM_service)        
            vector_store = Chroma(
                persist_directory = LOCAL_VECTOR_STORE_DIR.as_posix() + "/" + vectorstore_name,
                embedding_function=embeddings
            )
            
        # 6. base retriever: Vector store-backed retriever 
        base_retriever = Vectorstore_backed_retriever(
            vector_store,
            search_type=base_retriever_search_type,
            k=base_retriever_k,
            score_threshold=base_retriever_score_threshold
        )
        retriever = None
        if retriever_type=="Vectorstore_backed_retriever": 
            retriever = base_retriever
    
        # 7. Contextual Compression Retriever
        if retriever_type=="Contextual_compression":    
            retriever = create_compression_retriever(
                embeddings=embeddings,
                base_retriever=base_retriever,
                k=compression_retriever_k,
            )
    
        # 8. CohereRerank retriever
        if retriever_type=="Cohere_reranker":
            retriever = CohereRerank_retriever(
                base_retriever=base_retriever, 
                cohere_api_key=cohere_api_key, 
                cohere_model=cohere_model, 
                top_n=cohere_top_n
            )
    
        print(f"\n{retriever_type} is created successfully!")
        print(f"Relevant documents will be retrieved from vectorstore ({vectorstore_name}) which uses {LLM_service} embeddings \
and has {vector_store._collection.count()} chunks.")
        
        return retriever
    except Exception as e:
        print(e)

## ChatModel

In [26]:
# LLM Model
llm = ChatOllama(model="mistral:7b-instruct-v0.2-q8_0")

In [28]:
import torch
# Message test
messages = [
    ChatMessage(role="control", content="thinking"),
    HumanMessage("What is 3^3?"),
]


torch.cuda.empty_cache()
response = llm.invoke(messages)
print(response.content)

 The expression 3 raised to the power of 3, or 3 cubed, can be calculated as follows:

3 * 3 * 3 = 27

So, the result of 3 raised to the power of 3 is 27.
